Estamos mal de recall, incluso usando el nuevo assistant api con retrieval de openai no puedo sacar toda la data. no es confiable la extraccion para una empresa.

In [7]:
"""PDF loaders"""
import logging
import os
import uuid
import json
import requests
import tiktoken
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)
from langchain.document_loaders import (
    PyPDFLoader,
    PDFMinerPDFasHTMLLoader,
    UnstructuredMarkdownLoader,
)
from dotenv import load_dotenv
from markdownify import markdownify as md




load_dotenv()

OPENAI_API_TYPE = "openai"


embeddings = OpenAIEmbeddings()


tokenizer = tiktoken.get_encoding("cl100k_base")


def tiktoken_len(text):
    """Returns the length of a text in tokens."""
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)


async def download_pdf_file_and_ingest(resource):
    """
    Downloads a pdf file and ingests it into the index
    """
    print(f"[[download_pdf_file_and_ingest]] starting ...")
    url = resource["url"]
    index = resource["index"]
    chunk_size = 200
    chunk_overlap = 20
    metadata = {"url": url, "index": index, "type": "pdf"}
    id = str(uuid.uuid1())

    local_path = f"./resources/{index}-{str(uuid.uuid1())}.pdf"

    response = requests.get(url)

    with open(local_path, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully to {local_path} !")

    try:
        doc = PyPDFLoader(local_path).load()
        # text_splitter = CharacterTextSplitter()
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=tiktoken_len,
            separators=["\n\n\n\n\n", "\n\n\n", "\n\n"],
        )
        chunks = text_splitter.split_documents(doc)
        faiss_db = await load_or_create_faiss_db(
            index_name=index, embeddings=embeddings
        )
        metadatas = [metadata] * len(chunks)
        texts = []
        for chunk in chunks:
            # dump the metadata to string
            metadata_text = json.dumps(metadata)
            text = metadata_text + "\n\n" + chunk.page_content
            texts.append(text)

        faiss_db.add_texts(
            texts=texts,
            metadatas=metadatas,
        )

        faiss_db.save_local(f"indexes/{index}")
        os.remove(local_path)
        print(f"Index '{index}' has a new document ")
        return {"status": "success"}

    except Exception as e:
        print(f"Error while ingesting pdf {e}")
        return


async def download_pdf_md_and_ingest(resource):
    """
    Downloads a pdf file, converts it to markdown and ingests it into the index
    """
    print(f"[[download_pdf_file_and_ingest]] starting ...")
    url = resource["url"]
    index = resource["index"]
    chunk_size = 200
    chunk_overlap = 20
    metadata = {"url": url, "index": index, "type": "pdf"}
    id = str(uuid.uuid1())

    local_path = f"./resources/{index}-{str(uuid.uuid1())}.pdf"

    response = requests.get(url)

    with open(local_path, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully to {local_path} !")

    try:
        html_document = PDFMinerPDFasHTMLLoader(local_path).load()[0]
        markdown = md(html_document.page_content)
        # write markdown to a temporary file
        with open("temp.md", "w", encoding="utf-8") as file:
            file.write(markdown)
        doc = UnstructuredMarkdownLoader("./temp.md").load()
        # text_splitter = CharacterTextSplitter()
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=tiktoken_len,
            separators=["\n\n\n\n\n", "\n\n\n", "\n\n"],
        )
        chunks = text_splitter.split_documents(doc)
        faiss_db = await load_or_create_faiss_db(
            index_name=index, embeddings=embeddings
        )
        metadatas = [metadata] * len(chunks)
        texts = []
        for chunk in chunks:
            # dump the metadata to string
            metadata_text = json.dumps(metadata)
            text = metadata_text + "\n\n" + chunk.page_content
            texts.append(text)

        faiss_db.add_texts(
            texts=texts,
            metadatas=metadatas,
        )

        faiss_db.save_local(f"indexes/{index}")
        os.remove(local_path)
        print(f"Index '{index}' has a new document ")
        return {"status": "success"}

    except Exception as e:
        print(f"Error while ingesting pdf {e}")
        return

AttributeError: module 'openai' has no attribute 'Embedding'

Aca pruebo una funcion nueva en la que en vez de un resumen pruebo con una lista de oraciones.


In [30]:
import openai
import os
import json

openai.api_key = os.environ["OPENAI_API_KEY"]
MODEL = "gpt-4-1106-preview"


def clean_data(text: str):
    """Transform text using OpenAI's API."""
    function_description = f"""
    The user will give you a credit card consumption report with some meaningful data surrounded by meaningless data.
    Extract all meaningful information from the document and create a list of sentences, put those in the 'sentences' field.
    Then create a summary of the consumptions and due payments, put that summary in the summary field.
    Use this format:
    'The user [did/bought/used/paid/...] [something] on [date] at [place/store] for [amount of money].'
    Then extract the name of the user and put it in the user field.
    Then extract the date of the document and put it in the date field.
    No important details should be left out.
    This is very important information for the user, so please be sure to include all the relevant information.
    """
    tools = [
        {"type": "function",
         "function":{
            "name": "default",
            "description": function_description,
            "parameters": {
                "type": "object",
                "properties": {
                    "summary": {
                        "type": "string",
                        "description": "The summary of the document according to the function description.",
                    },
                    "sentences": {
                        "type": "string",
                        "description": "The sentences extracted of the document according to the function description.",
                    },
                    "user": {
                        "type": "string",
                        "description": "[User's first name] [User's last name]",
                    },
                    "date": {
                        "type": "string",
                        "description": "Date in the format YYYY-MM-DD",
                    },
                },
            },
        }
        }
    ]
    messages = [
        {"role": "user", "content": text},
    ]
    try:
        completion = openai.ChatCompletion.create(
            model=MODEL,
            messages=messages,
            tools=tools,
            tool_choice={"type":"function", "function": {"name": "default"}},
        )
        print(completion)
        arguments = json.loads(
            str(completion.choices[0])
        )

        return arguments

    except openai.error.InvalidRequestError:
        print("OpenAI request failed")
        return None

In [31]:
summary = clean_data(text=context)

{
  "id": "chatcmpl-8IcFnieEyYLFfDkY9PAs5Dy0j9mwp",
  "object": "chat.completion",
  "created": 1699447643,
  "model": "gpt-4-1106-preview",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "tool_calls": [
          {
            "id": "call_Foh5CuSDOrXTCYwha5iAammL",
            "type": "function",
            "function": {
              "name": "default",
              "arguments": "{\n  \"summary\": \"Alexander Ditzend used his Mastercard Platinum credit card for multiple transactions between September and October 2023. The card was charged for purchases at Scape Park, Autopistas, Don Web, Adobe, and various other vendors.\",\n  \"sentences\": \"The user bought access to Scape Park on 24-Sep-23 for ARS 2,603.00. The user paid toll charges on 08-Sep-23 at Autopistas del 10/18. The user made payments to DON WEB on 14-Sep-23 for undisclosed amounts. The user made a purchase at PPRO *Adobe on 22-Sep-23. The user made

In [21]:
type(summary)

dict

In [22]:
summary

{'summary': 'The user Alexander Ditzend had a total payment due of ARS 102,041.04 for his Mastercard Platinum credit card, with transactions spanning from 24-Sep-23 to 04-Oct-23, including purchases at Scape Park, toll payments, web services from Don Web, Adobe subscription, transactions at Flybondi.com and with Claro.',
 'sentences': "The user Alexander Ditzend paid ARS 102,041.04 on various dates for services which include: 'Scape Park' on 24-Sep-23, toll on 08-Sep-23 and 22-Sep-23, 'Don Web' services on 14-Sep-23, 'Adobe' subscription on 22-Sep-23, transaction at 'Flybondi.com' on 03-Oct-23, and 'Claro' service on 04-Oct-23.",
 'user': 'Alexander Ditzend',
 'date': '2023-09-24'}

In [10]:
summary['date']


'2023-09-07'

In [19]:
summary['sentences']

KeyError: 'sentences'

In [33]:
summary['user']

'ALEXANDER DITZEND'

Paso siguiente: guardar el resumen en un indice junto con 'user' y 'date' como metadata ademas de 'source'. Que eso este disponible para hacer QA y generar preguntas de prueba con gpt4 para evaluar la capacidad.

Pero antes hay que tener un mejor metodo con recall completo de los movimientos, los precios y la moneda. 


Pruebo con otro método dentro de la función.

In [ ]:
import openai
import os
import json

openai.api_key = os.environ["OPENAI_API_KEY"]
MODEL = "gpt-4-1106-preview"


def clean_data_2(text: str):
    """Transform text using OpenAI's API."""
    function_description = f"""
    The user will give you a credit card consumption report with some meaningful data surrounded by meaningless data.
    Extract all meaningful information from the document and create a list of sentences, put those in the 'sentences' field.
    Then create a summary of the consumptions and due payments, put that summary in the summary field.
    Use this format:
    'The user [did/bought/used/paid/...] [something] on [date] at [place/store] for [amount of money].'
    Then extract the name of the user and put it in the user field.
    Then extract the date of the document and put it in the date field.
    No important details should be left out.
    This is very important information for the user, so please be sure to include all the relevant information.
    """
    tools = [
        {
            "type": "function",
            "function": {
                "name": "default",
                "description": function_description,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "summary": {
                            "type": "string",
                            "description": "The summary of the document according to the function description.",
                        },
                        "sentences": {
                            "type": "string",
                            "description": "The sentences extracted of the document according to the function description.",
                        },
                        "user": {
                            "type": "string",
                            "description": "[User's first name] [User's last name]",
                        },
                        "date": {
                            "type": "string",
                            "description": "Date in the format YYYY-MM-DD",
                        },
                    },
                },
            },
        }
    ]
    messages = [
        {"role": "user", "content": text},
    ]
    try:
        completion = openai.ChatCompletion.create(
            model=MODEL,
            messages=messages,
            tools=tools,
            tool_choice={"type": "function", "function": {"name": "default"}},
        )
        print(completion)
        arguments = json.loads(str(completion.choices[0]))

        return arguments

    except openai.error.InvalidRequestError:
        print("OpenAI request failed")
        return None